In [62]:
# Importing modules

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import warnings
warnings.filterwarnings("ignore")

In [63]:
# Reading training data and dropping time column

pd.set_option('display.max_columns', None)
df = pd.read_csv('dataset/train/training-data.csv')
df = df.drop(['time'], axis=1)
df

,f1_c,f1_a,f1_s,f1_d,f2_c,f2_a,f2_s,f2_d,prg_c,prg_a,prg_s,prg_d,prd_c,prd_a,prd_s,prd_d,pr_s,pr_d,lq_s,lq_d,cmp_a_s,cmp_a_d,cmp_b_s,cmp_b_d,cmp_c_s,cmp_c_d
0,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
1,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
2,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
3,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
4,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9780,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0
9781,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0
9782,42919,53824,65535,0,1868,1895,161,-2,0,0,0,0,52984,51424,21841,692,55163,60,28991,-4,30828,0,980,0,33725,0
9783,42919,53824,65535,0,1868,1895,161,0,0,0,0,0,52984,51424,21841,0,55163,0,28991,0,30828,0,980,0,33725,0


In [64]:
# Function that computes mean and variance for features

def compute_mu_and_variance(df):
    mu_arr = []
    variance_arr = []
    for column in df:
        arr = df[column].to_numpy()
        mu = np.average(arr)
        std = np.sqrt(np.mean((arr - np.mean(arr)) ** 2))
        variance = np.mean((arr - np.mean(arr)) ** 2)
        mu_arr.append(mu)
        variance_arr.append(variance)
    return mu_arr, variance_arr

In [65]:
# Applying guassian model and dropping unimportant features

def guassian_proabability(df, mu_arr, variance_arr):
    count = 0
    sub_df = df.iloc[:, :27]
    for col in sub_df:
        prob_list = []
        arr = sub_df[col].to_numpy()
        for i in arr:
            prob = (1/(np.sqrt(variance_arr[count]) * np.sqrt(2*np.pi)))*np.e**(-((i - mu_arr[count])**2)/(2*variance_arr[count]))
            prob_list.append(prob)
        df[list(df.columns)[count] + '_prob'] = prob_list
        count += 1
    df = df.drop(columns=['f1_c_prob', 'f1_a_prob', 'f1_s_prob', 'f1_d_prob', 'f2_c_prob', 'f2_a_prob', 'f2_s_prob', 'prg_a_prob', 'prg_c_prob', 'prg_s_prob', 'prd_c_prob', 'prd_a_prob', 'prd_s_prob', 'pr_s_prob', 'lq_s_prob', 'cmp_a_s_prob', 'cmp_b_s_prob', 'cmp_c_s_prob'])
    sub_prod_df = df.iloc[:,26:35]
    df['p_of_x'] = sub_prod_df['f2_d_prob'] * sub_prod_df['prg_d_prob'] * sub_prod_df['prd_d_prob'] * sub_prod_df['pr_d_prob'] * sub_prod_df['lq_d_prob'] * sub_prod_df['cmp_a_d_prob'] * sub_prod_df['cmp_b_d_prob'] * sub_prod_df['cmp_c_d_prob']
    return df

In [67]:
# Getting mean, variance from features and applying the model to get probabilities

mu_arr, variance_arr = compute_mu_and_variance(df)
df = guassian_proabability(df, mu_arr, variance_arr)
df

,f1_c,f1_a,f1_s,f1_d,f2_c,f2_a,f2_s,f2_d,prg_c,prg_a,prg_s,prg_d,prd_c,prd_a,prd_s,prd_d,pr_s,pr_d,lq_s,lq_d,cmp_a_s,cmp_a_d,cmp_b_s,cmp_b_d,cmp_c_s,cmp_c_d,f2_d_prob,prg_d_prob,prd_d_prob,pr_d_prob,lq_d_prob,cmp_a_d_prob,cmp_b_d_prob,cmp_c_d_prob,p_of_x,f2_d_prob_prob
0,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
1,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
2,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
3,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
4,0,0,0,0,1851,1831,155,0,0,0,0,0,50522,51872,21637,0,53281,0,28700,0,30781,0,1186,0,33567,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9780,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
9781,53824,61269,65535,0,1896,1923,163,0,0,0,0,0,51424,49824,21149,0,55103,0,28995,0,30828,0,980,0,33725,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431
9782,42919,53824,65535,0,1868,1895,161,-2,0,0,0,0,52984,51424,21841,692,55163,60,28991,-4,30828,0,980,0,33725,0,0.000126,0.00007,1.947107e-07,0.001216,0.018768,1.278333,1.508892,2.762855,2.096817e-19,0.003282
9783,42919,53824,65535,0,1868,1895,161,0,0,0,0,0,52984,51424,21841,0,55163,0,28991,0,30828,0,980,0,33725,0,0.836427,0.00007,2.508183e-03,0.014941,0.219541,1.278333,1.508892,2.762855,2.567805e-09,1.759431


In [68]:
# Finding the best threshold value that maximizes F1 score

def find_best_epsilon(y_label, prob):
    
    F1 = 0
    bestF1 = 0
    bestEpsilon = 0
    
    stepsize = (np.max(prob) - np.min(prob))/1000
        
    epsVec = np.arange(np.min(prob), np.max(prob), stepsize)
    noe = len(epsVec)
    
    for eps in range(noe):
        epsilon = epsVec[eps]
        pred = (prob < epsilon)
        prec, rec = 0,0
        tp,fp,fn = 0,0,0
        
        try:
            for i in range(np.size(prob,0)):
                if pred[i] == 1 and y_label == 1:
                    tp+=1
                elif pred[i] == 1 and y_label == 0:
                    fp+=1
                elif pred[i] == 0 and y_label == 1:
                    fn+=1
            prec = tp/(tp + fp)
            rec = tp/(tp + fn)
            F1 = 2*prec*rec/(prec + rec)
            if F1 > bestF1:
                bestF1 = F1
                bestEpsilon = epsilon
        except ZeroDivisionError:
            continue          
       
    return bestF1, bestEpsilon

In [72]:
# Preparing cross validation data for tuning

cv_dir = 'dataset/valid'
valid_filenames = glob.glob(cv_dir + "\*.csv")
counter = 0
y_label = [1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0]
f1_and_epsilon_list = []
curr_best_f1 = 0
best_f1 = 0
best_epsilon = 0

for i in valid_filenames:
    # Went through all the data originally. Just capping it to 10 files for the demo since it takes so long
    # to run and it is discovered that best epsilon is within first 10 rows 
    if counter < 11:
        print(counter)
        cv_df = pd.read_csv(i)
        cv_df = cv_df.drop(['time'], axis=1)
        mu_arr, variance_arr = compute_mu_and_variance(cv_df)
        cv_df = guassian_proabability(cv_df, mu_arr, variance_arr)
        arr = cv_df['p_of_x'].to_numpy()
        curr_best_f1, curr_epsilon = find_best_epsilon(y_label[counter], arr)
        f1_and_epsilon_list.append([curr_best_f1, curr_epsilon])
        best_f1 = max(best_f1, curr_best_f1)
        print('curr epsilon: ', curr_epsilon)
        print('curr f1: ', curr_best_f1)
        print('best f1: ', best_f1)
        counter += 1
    
for i in f1_and_epsilon_list:
    if i[0] == best_f1:
        best_epsilon = i[1]

print('Best epsilon is: ' + str(best_epsilon))

0
curr epsilon:  2.51655008662266e-09
curr f1:  0.3533070005882847
best f1:  0.3533070005882847
1
curr epsilon:  2.5158649504621003e-09
curr f1:  0.9999489665731054
best f1:  0.9999489665731054
2
curr epsilon:  0
curr f1:  0
best f1:  0.9999489665731054
3
curr epsilon:  2.51655008662266e-09
curr f1:  0.3533070005882847
best f1:  0.9999489665731054
4
curr epsilon:  0
curr f1:  0
best f1:  0.9999489665731054
5
curr epsilon:  2.488979828656634e-09
curr f1:  0.35247624653157317
best f1:  0.9999489665731054
6
curr epsilon:  2.3901472405847056e-09
curr f1:  0.20311926605504588
best f1:  0.9999489665731054
7
curr epsilon:  2.1880743609039242e-09
curr f1:  0.2079253225954059
best f1:  0.9999489665731054
8
curr epsilon:  1.91873300436977e-09
curr f1:  0.205607476635514
best f1:  0.9999489665731054
9
curr epsilon:  2.5158649504621003e-09
curr f1:  0.9999489665731054
best f1:  0.9999489665731054
10
curr epsilon:  2.5281328563360076e-09
curr f1:  0.35577568838146406
best f1:  0.9999489665731054
Be

In [38]:
eps = 2.5158649504621003e-09

In [74]:
# Running the model against test dataset after getting best epsilon

test_dir = 'dataset/test'
test_filenames = glob.glob(test_dir + "\*.csv")
pred_list = []

counter = 0
for i in test_filenames:
    curr_pred_list = []
    test_df = pd.read_csv(i)
    test_df = test_df.drop(['time'], axis=1)
    mu_arr, variance_arr = compute_mu_and_variance(test_df)
    test_df = guassian_proabability(test_df, mu_arr, variance_arr)
    arr = test_df['p_of_x'].to_numpy()
    for i in arr:
        if i < best_epsilon:
            curr_pred_list.append(1)
        else:
            curr_pred_list.append(0)
    pred_list.append(curr_pred_list)
    print('Done with file ' + str(counter))
    counter += 1
    

Done with file 0
Done with file 1
Done with file 2
Done with file 3
Done with file 4
Done with file 5
Done with file 6
Done with file 7
Done with file 8
Done with file 9
Done with file 10
Done with file 11
Done with file 12
Done with file 13
Done with file 14
Done with file 15
Done with file 16
Done with file 17
Done with file 18
Done with file 19
Done with file 20
Done with file 21
Done with file 22
Done with file 23
Done with file 24
Done with file 25
Done with file 26
Done with file 27
Done with file 28
Done with file 29
Done with file 30
Done with file 31
Done with file 32
Done with file 33
Done with file 34
Done with file 35
Done with file 36
Done with file 37
Done with file 38
Done with file 39
Done with file 40
Done with file 41
Done with file 42
Done with file 43
Done with file 44
Done with file 45
Done with file 46
Done with file 47
Done with file 48
Done with file 49
Done with file 50
Done with file 51
Done with file 52
Done with file 53
Done with file 54
Done with file 55
Do

In [75]:
# Writing the results to submission txt file and also printing the results here

import collections
c = 0
content = ''
for i in pred_list:
    frequency = collections.Counter(i)
    print(str(c) + ' ' + str(max(dict(frequency))))
    content += str(c) + ' ' + str(max(dict(frequency)))
    c += 1
file = open('submission.txt', 'w')
file.write(content)
file.close()

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 0
10 1
11 1
12 1
13 0
14 1
15 0
16 1
17 0
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 0
28 1
29 0
30 1
31 0
32 0
33 1
34 0
35 1
36 1
37 0
38 1
39 1
40 1
41 1
42 0
43 1
44 1
45 1
46 1
47 1
48 1
49 1
50 1
51 1
52 0
53 1
54 1
55 0
56 1
57 1
